#Using the WorldCat API to integrate holdings information and ILS data
## WorldCat API bit
How to query the bib-summary-holdings endpoint with Oauth2 tokens. (Python version using a Google Colab notebook)

## Loading in our Python libraries
If you've never used Python notebooks, this might be something new. You have the ability to break up your code into blocks. This makes it nice to not only organize code but it saves you from re-executing code blocks that only need to be run once. In the case of Python, we don't really need to load libraries and setup global things over and over in a notebook because they are persistent.

So, it's common to set up a block with your libraries and other things that only need to be run once. We only need to "press play" on this once per session.

I'm using the requests library do handle all of the HTTP GET requests.
Pandas is a commonly used library for handling table-like data.
I'm using the google.colab library to access my Google Drive to get and save files.
The csv library does what's "on the tin", allows you to work with CSV files.
Datetime allows you to easily work with date and time fields.


In [ ]:
# Code for processing Worldcat API

# Python "requests", simple HTTP library. https://pypi.org/project/requests/
import requests

# Pandas data analysis library
import pandas as pd

# Allow access to files and folders in google drive
from google.colab import files
from google.colab import drive

# CSV library for importing and expoerting to CSV
import csv

# Datetime library for handling dates and times
import datetime

# Mount the google drive for access in the code
# Note: Google will ask for permission to access the google drive
drive.mount('/drive')



Mounted at /drive


## Tokens
As I mention in the comments below, the bib-summary-holdings endpoint works on a token system. This allows for increased security but is a bit of a speed bump when you're trying to use the endpoint to retrieve a lot of records.

Have a look at the inline comments but the TL;DR is that this function gets a security token and then renews the token as needed.

Since this is a function we only need to "press play" on this once per session as well. Note: if we do make any changes, we will have to run it again for the changes to take effect.

In [ ]:
#getToken - Function for fetching and refreshing a token from Worldcat. Tokens are required for v2 API endpoints.
#This is called to get a new token at the start and refresh the token as needed.
#Note: The WorldCat API seems to be built for single transactions to the token timeout is fairly short.

def getToken(refreshTkn=None):

    # Set up the base URL and credentials. These creds are Ron's
    tokenUrl = 'https://oauth.oclc.org/token?'
    client_id = '[Worldcat Client ID goes here]'
    client_secret = '[Worldcat Client Secret goes here]'

    # First check if this is already a refresh token. If not, we're generating a new one.
    if refreshTkn is None:

        # Append the correct parameters to the URL, post the request, grab and assign the token info we need.
        tokenUrl = tokenUrl + 'scope=wcapi+refresh_token&grant_type=client_credentials'
        tokenResp = requests.post(tokenUrl, auth=(client_id, client_secret))
        tokenJson = tokenResp.json()
        apiToken = tokenJson["access_token"]
        refreshToken = tokenJson["refresh_token"]
        tokenExp = tokenJson["expires_at"]
        return apiToken, refreshToken, tokenExp;

    # Ok, this IS a refresh token.
    else:

        # Different parameters for a refresh token. Again, assign parameters, post request and assign the info returned.
        tokenUrl = tokenUrl + 'refresh_token=' + refreshTkn + '&grant_type=refresh_token'
        tokenResp = requests.post(tokenUrl, auth=(client_id, client_secret))
        tokenJson = tokenResp.json()
        apiToken = tokenJson["access_token"]
        refreshToken = tokenJson["refresh_token"]
        tokenExp = tokenJson["expires_at"]
        return apiToken, refreshToken, tokenExp;









## Getting the Data
Again, lots more technical details in the code comments but this is the main bit of code that:
- Fetches the OCLC numbers from a CSV file
- Generates the initial token
- Loops through the OCLC numbers we want to query the API
- Look through the data returned by the API and get the specific stuff we want
- Along the way, watch the time and refresh the token as needed
- Finally, write all of our data out to another CSV

Note: This is the block of code that we may run over and over again as we make changes to the data or code.



In [ ]:
# Main bit of the code

# Open up the input CSV file in the google drive. I tend to save everything in the Colab Notebooks folder just
# for ease of access but files can be saved anywhere. The "root" of the google drive is /drive/My Drive/
# read_csv is a pandas function that parses a CSV and allows you to assign it to a data frame
data = pd.read_csv('/drive/My Drive/Colab Notebooks/oclc_numbers.csv')

# drop_duplicates removes duplicate rows from the dataframe. I assign this to a new data frame
# so I can access the original data if needed.
data2 = data.drop_duplicates()

# Create a list variable for output
output = list()

# Get our initial token and grab the info we need to refresh when needed.
aToken, rToken, tExp = getToken()

# The main loop. Iterate through the rows that we got from the CSV
for index, row in data2.iterrows():

    # This block of code compares the expiration time on the token to the current time
    # If the expiration is less than a minute away, refresh the token

    # Find the current data and time.
    now = datetime.datetime.now()

    # Add a minute to it. Not sure why I called the variable twoMin
    twoMin = now + datetime.timedelta(minutes=1)

    # Convert the token expiration date to the same format as the current date ("now")
    exp = datetime.datetime.strptime(tExp, '%Y-%m-%d %H:%M:%SZ')

    # Check if token expiration time is less than 1 minute away. If it is, call getToken()
    # to renew the token
    if (twoMin>exp):
        aToken, rToken, tExp = getToken(rToken)

    # Grab the next oclc numeber out of the correct column (Make sure the column name in the CSV matches what's in the row identifier)
    oclcnum = str(row['OCLC Number (035a)'])

    # Create our URL with the oclc number inserted
    url = 'https://americas.discovery.api.oclc.org/worldcat/search/v2/bibs-summary-holdings?&oclcNumber=' + oclcnum + '&holdingsAllEditions=false&preferredLanguage=eng'

    # Run a http get request with the ULR we just created
    response = requests.get(url, headers={'Authorization': 'Bearer ' + aToken})

    # Commented out print statement for troubleshooting
    # print(index)

    # Grab the json data returned from the get request
    jsonData = response.json()

    # Drill down through the JSON structure to find the count we're looking for
    count = jsonData["briefRecords"][0]["institutionHolding"]["totalHoldingCount"]

    # Commented out print statement for troubleshooting
    #print(jsonData)

    # Create a comma separated row of our data. In this case the oclc number and count we just got
    rowData = [oclcnum, count]

    # Append the data to our output list
    output.append(rowData)

# Create a new data frame with out output list. Add column headings
df = pd.DataFrame(output, columns=["oclcnum","totalHoldingCount"])

# Output the dataframe to a CSV on our google drive. Output parameter of QUOTE_ALL. This means that the CSV
# will put double quotes around all of the data so it's all treated as text when importing. Sometimes long
# OCLC numbers (as well as other IDs, like MMSIDs) are mistaken for large numbers and converted to exponetial
# notation which tends to mess up IDs in Excel. Easier to treat it all as text and chenge to numbers as needed
# once it's in excel.
df.to_csv('/drive/My Drive/Colab Notebooks/oclc_holdingcounts_20230207.csv', quoting=csv.QUOTE_ALL)

